In [1]:
import pandas as pd

# Estimating living biomass stock changes from National Forest Inventory data

## 1. Introduction

This analysis is related to supplementary text 5 which compares our analysis to the results from national forest inventory data from [Grassi et al. (2023)](https://essd.copernicus.org/articles/15/1093/2023/). The analysis contains 3 steps:

1. Estimate the change in carbon stocks in forests in carbon units from the data in Grassi et al. (2023)

2. Correct for the fraction of the estimate that is in living biomass from the data in [Grassi et al. (2022)](https://essd.copernicus.org/articles/14/4643/2022/).

3. Incorporate estimates of missing sink in unmanaged forests from Grassi et al. (2023).

## 2. Load data

In [2]:
# load Table 5 from Grassi et al. 2022
grassi_2022 = pd.read_excel('../data/supplementary_analysis/NFI_data/essd-14-4643-2022-t05.xlsx',skiprows=2)

# load data from Grassi et al. 2023

grassi_2023 = pd.read_excel('../data/supplementary_analysis/NFI_data/National_inventories_LULUCF_data_2000-2020.xlsx',sheet_name='Table 5',skiprows=3)

## 3. Run analysis

### 3.1. Step 1 - get estimate from Grassi et al. 2023

In [3]:


grassi_rate = grassi_2023[(grassi_2023['LAND CATEGORY'].isin(['FOREST LAND','DEFORESTATION'])) & (grassi_2023['UNFCCC country']=='World')]['Av.. 2000-2020']

print(grassi_rate)
# CO2 to C
co2_c = (12/(12+16*2))
# convert to carbon 
grassi_rate_C = grassi_rate.sum() * co2_c

403   -6659.157064
610    4353.875362
Name: Av.. 2000-2020, dtype: object


### 3.2. Step 2 - extact data only for living biomass

Grassi et al. (2022) estimates that ≈300 MtCO<sub>2</sub> yr<sup>-1</sup> is sequestered in harvested wood products. In addition, We remove from the estimate the contribution of dead organic matter and mineral soils from Table 5 in Grassi et al. 2022.

In [4]:
HWP = 300 # 300 MtCO2 yr-1

non_biomass = grassi_2022.iloc[[2,4]][['Dead.1','Soil.2']].map(lambda x: x.strip('−')).astype(float).sum().sum()

non_biomass_tot = (non_biomass + HWP) * co2_c

In [5]:
# remove from the accumulation in Grassi the non-biomass part and convert to GtC yr-1
NFI_estimate = -(grassi_rate_C + non_biomass_tot)/1e3

print(f'Our estimate for NFI based forest living biomass sink is: {NFI_estimate:.1f} GtC yr-1')

Our estimate for NFI based forest living biomass sink is: 0.4 GtC yr-1
